Google Colab Installation of mlinphysics

In [3]:
COLAB_FOLDER = 'MLP' # change as needed
GITHUB_USER  = 'hbprosper'
GITHUB_REPO  = 'mlinphysics'
GITHUB_FOLDERS = ['mlinphysics']
#------------------------------------------------------
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    print('\nGoogle Drive mounted\n')
    IN_COLAB = True
except:
    print('\nRunning locally\n')
    IN_COLAB = False

if IN_COLAB:
    MYDRIVE     = '/content/gdrive/MyDrive'
    GITHUB_BASE = 'https://raw.githubusercontent.com'
    MAIN        = 'refs/heads/main'
    GITHUB_PATH = f'{MYDRIVE}/{COLAB_FOLDER}'
    #------------------------------------------------------
    %cd {GITHUB_PATH}
    %rm -f {GITHUB_PATH}/clone2colab.ipynb
    !wget -q {GITHUB_BASE}/{GITHUB_USER}/{GITHUB_REPO}/{MAIN}/clone2colab.ipynb
    %run {GITHUB_PATH}/clone2colab.ipynb

    %pip install torch_geometric


Running locally



Importing the required modules

In [4]:
# standard system modules
import os, sys

# standard module for tabular data
import pandas as pd

# standard module for array manipulation
import numpy as np

# standard module for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt

# standard research-level machine learning toolkit from Meta (FKA: FaceBook)
import torch
import torch.nn as nn
import torch.utils.data as dt

# module to access data in Hieracrchical Data Format (HDF or H5 format)
import h5py

# module to plot pixelized images
import imageio.v3 as im

# module to reimport Python modules
import importlib

# module for saving/loading serialized Python objects
import joblib

# module for shell utilities
import shutil

# ML in physics module
import mlinphysics.nn as mlp
import mlinphysics.utils.data as dat
import mlinphysics.utils.monitor as mon





Computational Device

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'\n\tAvailable device: {str(DEVICE):4s}\n')


	Available device: cpu 



Loading the data sets

In [6]:
produced_photons_array = np.loadtxt('scan_CsI_Se82_master_produced_photon.dat')
absorbed_photons_array = np.loadtxt('scan_CsI_Se82_master_absorbed_photon.dat')
observed_photon_array_array = np.loadtxt('scan_CsI_Se82_master_observed_photon_array.dat')
observed_photon_total_array = np.loadtxt('scan_CsI_Se82_master_observed_photon_total.dat')
beam_position_array = np.loadtxt('scan_CsI_Se82_master_beam_position.dat')

#Normalizing each PMT of the array to the total number of photons seen by the PMTs
normalized_PMT_array = np.zeros((1296,16))

for i in range (1296):
    for m in range (16):
        normalized_PMT_array[i][m] = observed_photon_array_array[i][m] / observed_photon_total_array[i]

print (normalized_PMT_array.shape)


(1296, 16)


Separating the data sets into training, validating, and testing

In [16]:
# generates a random sequence of integers from 0-1295
randomized_indices = np.random.permutation(1296)

training_indices = randomized_indices[0:432]
validation_indices = randomized_indices[432:864]
evaluating_model_indicies = randomized_indices[864:1296]

# training, validation, and evaluation datasets
training_photon_arrays = np.zeros((432,16))
validation_photon_arrays = np.zeros((432,16))
evaluation_photon_arrays = np.zeros((432,16))

for i in range (432):
    training_photon_arrays[i][:] = normalized_PMT_array[training_indices[i]][:]
    validation_photon_arrays[i][:] = normalized_PMT_array[validation_indices[i]][:]
    evaluation_photon_arrays[i][:] = normalized_PMT_array[evaluating_model_indicies[i]][:]

#reshaping the normalized data so that we have 432 images of 4x4 pixels
training_photons = training_photon_arrays.reshape(432, 4, 4)
validation_photons = validation_photon_arrays.reshape(432, 4, 4)
evaluating_photons = evaluation_photon_arrays.reshape(432, 4, 4)

print('training idices: ',training_indices)
print('validation indices: ',validation_indices)



training idices:  [ 228  589 1271  857  509  398  386  266  776  911 1289 1252  225  716
   81  891  298  534  612  865 1103  890  169  542  179 1031  177 1026
  456  392  296 1071   89  448 1100  400  507  215  562 1259   70  995
  744  399  931  477  710  133  944  706  649  918  337  805    2  265
 1125  598  476 1072  491  331  607  349  987  642  605  482 1270  699
  894  344  632  397 1067  802  688 1150 1180  464  898  366  167  801
  644  378 1005  924  269 1019  199  746  573 1157  207  557  439  143
  544  367 1146  203  526  192  478 1114  425  451  528  246  334  861
  631  621 1287  892  142  243  516  340  525   88 1168  403  511  702
 1135  377 1266  137  972  844  840 1240   66  816  691  160 1248 1207
  515 1227  387  560   87  273   34  181 1282  856   43 1033  233  843
  368 1267  113  563  727 1119 1129  283  486    6 1112  574  743 1087
   16  136 1218  490 1029  821  635   73  497  806  380  479  583  555
   98  452  412  383 1269  487  591 1278  493  722 1070  65